<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/NRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che4.ipynb -O che4.ipynb
!pip install importnb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from importnb import Notebook
with Notebook(): 
    from che4 import Props

import numpy as np

In [3]:
p = Props(['Ethanol','Isopropanol','Water'])

### NRTL Activity Coefficient Model

$ln (\gamma_i) = \dfrac{ \sum\limits_{j=1}^{n}x_j \tau_{ji}G_{ji}}{\sum\limits_{k=1}^{n}x_kG_{ki}} + 
\sum\limits_{j=1}^{n} \dfrac{x_jG_{ij}}{\sum\limits_{k=1}^{n}x_k G_{kj}} \left( \tau_{ij} - 
\dfrac{\sum\limits_{m=1}^{n} x_m \tau_{mj} G_{mj}}{\sum\limits_{k=1}^{n} x_kG_{kj}} \right)$

$\tau_{ji} = A_{ji} + \dfrac{B_{ji}}{T} + C_{ji}ln(T)+D_{ji}T$

$G_{ji}=exp \left( - \alpha_{ji} \tau_{ji} \right)$

$\alpha_{ij}=\alpha_{ji}$

In [38]:
def NRTL_einsum(x,T):
    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*np.log(T) + p.NRTL_D*T
    G = np.exp(-p.NRTL_alpha*tau)
    xtauG=np.einsum('j,ji',x,tau*G)
    xG=np.einsum('k,ki',x,G)
    xtauGdivxG=xtauG/xG
    term2 = np.einsum('j,ij',x,G*(tau-xtauGdivxG)/xG[None,:])
    return np.exp(xtauGdivxG+term2)

In [39]:
def NRTL(x, T):
    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*np.log(T) + p.NRTL_D*T
    G = np.exp(-p.NRTL_alpha*tau)
    n = x.size
    xtauG = np.zeros(3)
    xG=np.zeros(3)
    term2 = np.zeros(3)
    for i in range(n):
        for j in range(n):
            xtauG[i]=xtauG[i]+x[j]*tau[j,i]*G[j,i]
        for k in range(n):
            xG[i]=xG[i]+x[k]*G[k,i]
    xtauGdivxG = xtauG/xG

    for i in range(n):
        for j in range(n):
            term2[i]=term2[i]+ x[j]*G[i,j]*(tau[i,j] - xtauGdivxG[j])/xG[j]
    return (np.exp(xtauGdivxG + term2))

        

In [40]:
x=np.array([1/3, 1/3, 1/3])

In [41]:
NRTL(x,298.)

array([1.02500286, 1.13152448, 2.06917874])

In [42]:
def NRTL_gamma(x, T):
    x=np.asarray(x).reshape((1,-1))
    tau = (p.NRTL_A + p.NRTL_B / T + p.NRTL_C * np.log(T) +
            p.NRTL_D * T)
    G = np.exp(-p.NRTL_alpha * tau)
    xG=x @ G
    xtauGdivxG = (x@ (tau*G)/ xG)
    lngamma = xtauGdivxG +  (((G*(tau - xtauGdivxG))/xG) @x.T).T
    return np.exp(lngamma).flatten()

In [43]:
NRTL_gamma(x,298.)


array([1.02500286, 1.13152448, 2.06917874])

In [44]:
NRTL_einsum(x,298.)

array([1.02500286, 1.13152448, 2.06917874])